###Initalizations

In [ ]:
%%capture
!pip install flask-ngrok
!pip install web3==4.9.1
import json
from web3 import Web3
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

In [ ]:
app = Flask(__name__, template_folder = '/content/drive/MyDrive/BrainBox/templates')
run_with_ngrok(app)

###Code

In [ ]:
class BrainBox():
    def __init__(self, network = 'rinkeby'):
        self.infura_url = f'https://{network}.infura.io/v3/953247d0c42b419aa3416810d625cc8c'
        self.chain_id_list = {'kovan': 42, 'rinkeby': 4, 'mainnet': 1, 'ropsten': 3}
        self.chain_id = self.chain_id_list[network]
        self.web3 = Web3(Web3.HTTPProvider(self.infura_url))
        assert self.web3.isConnected() == True

        self.abi = json.loads('[{"constant":true,"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"mint","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"}]')
        self.address = '0x4F96Fe3b7A6Cf9725f59d353F723c1bDb64CA6Aa'
        self.contract = self.web3.eth.contract(address = self.address, abi = self.abi)
                
        self.player_account = '0xDb28F5e9B299df1863834C601F91aBE508d8D826'
        self.quizmaster_account = '0xA1cc47d188D6E436E0Bae2f72cC936B89778f603'
        self.quizmaster_privatekey = '14fab9d51095fcf88e4c10704d26dcf4f2ac4bd189d9762925ad509fa58a7ea5'

        self.q_num = 0
        self.winnings = 0
        self.proceed = True
        self.q_asked = False
        self.x, self.y, self.z = True, True, True
        self.bank = {'Q1': {'Q': 'Q01 : Who is the father of Tony Stark?<br>&nbsp;&nbsp;A: Howard, B: Obadiah, C: Robert, D: Henry', 
                            'X': 'Your friend says A.',
                            'Y': 'A:63% B:22% C:13% D:02%.',
                            'Z': 'You are left with A and C.',
                            'C': 'A'},
                     'Q2': {'Q': 'Q02 : What is the last name of Wasp?<br>&nbsp;&nbsp;A: Pym, B: Lang, C: van Dyne, D: van Dyke', 
                            'X': 'Your friend says C.',
                            'Y': 'A:06% B:01% C:83% D:10%.',
                            'Z': 'You are left with B and C.',
                            'C': 'C'},
                     'Q3': {'Q': 'Q03 : What is the name of the hammer of Thor?<br>&nbsp;&nbsp;A: Megingjord, B: Stormcaster, C: Beta Ray, D: Mjolnir', 
                            'X': 'Your friend says D.',
                            'Y': 'A:01% B:02% C:06% D:91%.',
                            'Z': 'You are left with B and D.',
                            'C': 'D'},
                     'Q4': {'Q': 'Q04 : Which army fought the Avengers in New York?<br>&nbsp;&nbsp;A: Kree, B: Chitauri, C: Skrull, D: Askavarian', 
                            'X': 'Your friend says B.',
                            'Y': 'A:10% B:71% C:14% D:05%.',
                            'Z': 'You are left with A and B.',
                            'C': 'B'},  
                     'Q5': {'Q': 'Q05 : What is the last name of Mysterio?<br>&nbsp;&nbsp;A: Beck, B: Musk, C: Watson, D: Parker', 
                            'X': 'Your friend says A.',
                            'Y': 'A:36% B:28% C:24% D:12%.',
                            'Z': 'You are left with A and C.',
                            'C': 'A'},  
                     'Q6': {'Q': 'Q06 : Where is the new Avengers facility?<br>&nbsp;&nbsp;A: New York, B: Cleveland, C: Queens, D: Upstate', 
                            'X': 'Your friend says D.',
                            'Y': 'A:01% B:14% C:05% D:80%.',
                            'Z': 'You are left with C and D.',
                            'C': 'D'},  
                     'Q7': {'Q': 'Q07 : Who is the sister of Thor?<br>&nbsp;&nbsp;A: Valkyrie, B: Frigga, C: Hela, D: Natalie', 
                            'X': 'Your friend says C.',
                            'Y': 'A:13% B:22% C:63% D:02%.',
                            'Z': 'You are left with B and C.',
                            'C': 'C'},                                                               
                     'Q8': {'Q': 'Q08 : Who is the biological father of Loki?<br>&nbsp;&nbsp;A: Buri, B: Bor, C: Odin, D: Laufey', 
                            'X': 'Your friend says D.',
                            'Y': 'A:09% B:01% C:09% D:81%.',
                            'Z': 'You are left with A and D.',
                            'C': 'D'},                              
                     'Q9': {'Q': 'Q09 : What is the code of Bleeding Edge Armour?<br>&nbsp;&nbsp;A: Mark 92, B: Mark 64, C: Mark 85, D: Mark 50', 
                            'X': 'Your friend says A.',
                            'Y': 'A:07% B:08% C:06% D:79%.',
                            'Z': 'You are left with B and D.',
                            'C': 'D'},
                    'Q10': {'Q': 'Q10 : Where was the Soul Stone found?<br>&nbsp;&nbsp;A: Xandar, B: Vormir, C: Knowhere, D: Xylosia', 
                            'X': 'Your friend says B.',
                            'Y': 'A:01% B:99% C:00% D:00%.',
                            'Z': 'You are left with A and B.',
                            'C': 'B'},
                    'Q11': {'Q': 'Q11 : Who knew of the location of Soul Stone?<br>&nbsp;&nbsp;A: Gamora, B: Mantis, C: Thor, D: Algrim', 
                            'X': 'Your friend says A.',
                            'Y': 'A:63% B:22% C:13% D:02%.',
                            'Z': 'You are left with A and C.',
                            'C': 'A'},  
                    'Q12': {'Q': 'Q12 : Who is the intern of Darcy?<br>&nbsp;&nbsp;A: Hewitt, B: Ian, C: Bosco, D: Kyle', 
                            'X': 'Your friend says B.',
                            'Y': 'A:10% B:71% C:14% D:05%.',
                            'Z': 'You are left with A and B.',
                            'C': 'B'},  
                    'Q13': {'Q': 'Q13 : How many futures did Doctor Strange see?<br>&nbsp;&nbsp;A: 14000605, B: 14000609, C: 17000209, D: 17000208', 
                            'X': 'Your friend says A.',
                            'Y': 'A:63% B:22% C:13% D:02%.',
                            'Z': 'You are left with A and C.',
                            'C': 'A'},  
                    'Q14': {'Q': 'Q14 : What is the weapon of Yondu?<br>&nbsp;&nbsp;A: Necroblade, B: Kylosia, C: Yaka, D: Sonic Cannon', 
                            'X': 'Your friend says C.',
                            'Y': 'A:13% B:22% C:63% D:02%.',
                            'Z': 'You are left with B and C.',
                            'C': 'C'},                                                               
                    'Q15': {'Q': 'Q15 : What is the compartment that housed Iron Spider?<br>&nbsp;&nbsp;A: 15P, B: 14D, C: 17A, D: 18C', 
                            'X': 'Your friend says C.',
                            'Y': 'A:06% B:01% C:83% D:10%.',
                            'Z': 'You are left with A and C.',
                            'C': 'C'},    
                     }


    def perform_transaction(self):
        nonce =  self.web3.eth.getTransactionCount(self.quizmaster_account)
        tx = {'from': self.quizmaster_account, 'to': self.player_account, 'value': (self.winnings) * (10 ** 18),  'gas': 1000000,  
              'gasPrice': self.web3.toWei(150,'gwei'), 'nonce': nonce,  'chainId': self.chain_id}
        signed_tx = self.web3.eth.account.signTransaction(tx, self.quizmaster_privatekey)
        tx_hash = self.web3.eth.sendRawTransaction(signed_tx.rawTransaction)


    def check_correct_answer(self, x):
        self.q_asked = False
        correct_answer = self.bank[f'Q{self.q_num + 1}']['C']
        if correct_answer.lower() == x.lower():
            self.winnings += ((self.q_num // 5) + 1)
            self.q_num += 1
            if self.q_num < 15:
                return 'Well done! Correct Answer!'
            else:
                self.proceed = False
                return f'Right answer again! You are a Millionaire!<br>&nbsp;&nbsp;Your final winnings are {self.winnings:018d} ETH.<br>&nbsp;&nbsp;The game quits now. Transaction has been initiated.<br>&nbsp;&nbsp;Your winnings will be credited to your address in a <br>&nbsp;&nbsp;while after the mining. Cheers!'
            
        else:
            self.proceed = False
            return f'Tough Luck!<br>&nbsp;&nbsp;Your final winnings are {self.winnings:018d} ETH.<br>&nbsp;&nbsp;The game quits now. Transaction has been initiated.<br>&nbsp;&nbsp;Your winnings will be credited to your address!<br>&nbsp;&nbsp;Cheers!'


    def display_lifelines(self, x):
        if x == 'x':
            if self.x:
                self.x = False
                return 'Phone a Friend : ' + self.bank[f'Q{self.q_num + 1}']['X']
            else:
                return 'Phone a Friend used.'
        elif  x == 'y':
            if self.y:
                self.y = False
                return 'Audience Poll : ' + self.bank[f'Q{self.q_num + 1}']['Y']
            else:
                return 'Audience Poll used.'
        elif  x == 'z':
            if self.z:
                self.z = False
                return '50/50 : ' + self.bank[f'Q{self.q_num + 1}']['Z']
            else:
                return '50/50 used.'


    def check_input(self, x):
        if x in 'abcd':
            resp = self.check_correct_answer(x) if self.q_asked else "You haven't been asked a question yet. Hit 'N'." 
        elif x == 'n':
            resp = self.bank[f'Q{self.q_num + 1}']['Q']
            self.q_asked = True
        elif x in 'xyz':
            resp = self.display_lifelines(x) if self.q_asked else "You haven't been asked a question yet. Hit 'N'."
        elif x == '':
            resp = 'Type something..'
        else:
            resp = 'That seems to be an invalid response. Try again!'
        return resp

###User Interface

In [ ]:
@app.route("/")
def index():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    user_input = request.args.get('msg')
    if bbx.proceed:
        return bbx.check_input(user_input.lower()[0])
    else:
        return 'Game Over!'

###Driver

In [ ]:
bbx = BrainBox('rinkeby')
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://80af-34-73-98-105.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [29/Oct/2021 14:52:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:52:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Oct/2021 14:53:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:04] "GET /get?msg=n HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:06] "GET /get?msg=a HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:07] "GET /get?msg=n HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:10] "GET /get?msg=c HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:11] "GET /get?msg=n HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:14] "GET /get?msg=d HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:16] "GET /get?msg=n HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:20] "GET /get?msg=b HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:22] "GET /get?msg=n HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:24] "GET /get?msg=a HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:26] "GET /get?msg=n HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 14:53:29] "GET /get?msg